<a href="https://colab.research.google.com/github/rajeshsurya59/Offensive-Language-Detection-BERT/blob/main/BERT_%20without_outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.28.0

In [ ]:
# Mounting Google Drive - Only for Google Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#============================ Loading required libraries ============================#

import warnings
warnings.filterwarnings("ignore")

#!pip install datasets

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import tensorflow as tf
import torch
from torch.utils.data import DataLoader, Dataset

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
#============================ Loading the dataset ============================#

#df = pd.read_csv('/content/drive/MyDrive/hate_offensive_data.csv')
df = pd.read_csv('/content/drive/MyDrive/text/hate_offensive_data.csv')
del df['Unnamed: 0']

df['label'] = np.where(df['class']==2, 0, 1)

# Class 0 - Appropriate (Safe)
# Class 1 - Inappropriate (Hateful or offensive)

df.head()

In [ ]:
#============================ Text Preprocessing ============================#

stop_words = set(stopwords.words('english'))
stop_words.add("rt") # adding rt to remove retweet in dataset

# Removing Emojis
def strip_entities(raw_text):
    entity_regex = r"&[^\s;]+;"
    text = re.sub(entity_regex, "", raw_text)
    return text

# Replacing user tags
def remove_mentions(raw_text):
    regex = r"@([^ ]+)"
    text = re.sub(regex, "", raw_text)
    return text

# Removing URLs
def remove_urls(raw_text):
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text = re.sub(url_regex, '', raw_text)
    return text

# Removing Unnecessary Symbols
def clean_symbols(raw_text):
    text = raw_text.replace('"', '')
    text = text.replace("'", '')
    text = text.replace("!", '')
    text = text.replace("`", '')
    text = text.replace("..", '')
    text = text.replace(".", '')
    text = text.replace(",", '')
    text = text.replace("#", '')
    text = text.replace(":", '')
    text = text.replace("?", '')
    return text

# Stemming
def stemming(raw_text):
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in raw_text.split()]
    return ' '.join(words)

# Removing stopwords
def filter_stopwords(raw_text):
    tokenize = word_tokenize(raw_text)
    text = [word for word in tokenize if not word.lower() in stop_words]
    text = ' '.join(text)
    return text

def preprocess(data):
    clean = []
    clean = [text.lower() for text in data]
    clean = [remove_mentions(text) for text in clean]
    clean = [strip_entities(text) for text in clean]
    clean = [remove_urls(text) for text in clean]
    clean = [clean_symbols(text) for text in clean]
    clean = [stemming(text) for text in clean]
    clean = [filter_stopwords(text) for text in clean]

    return clean

nltk.download('punkt_tab') # Download the missing NLTK resource

tweets = list(df['tweet'])
labels = list(df['label'])
clean_tweets = preprocess(tweets)

df_tweet = pd.DataFrame({'tweet': tweets, 'clean_tweet': clean_tweets, 'label': labels})

In [ ]:
#============================ Read Preprocessed Data ============================#

#df = pd.read_csv('/content/drive/MyDrive/preprocessed_data.csv')
df = pd.read_csv('/content/drive/MyDrive/text/preprocessed_data.csv')
df['clean_tweet'] = df['clean_tweet'].astype(str)
df['tweet'] = df['tweet'].astype(str)

tweets = list(df['tweet'])
clean_tweets = list(df['clean_tweet'])
labels = list(df['label'])

df

In [ ]:
df_tweet

In [ ]:
df_tweet['label'].value_counts()
# Class 0 - Appropriate (Safe)
# Class 1 - Inappropriate (Hateful or offensive)

In [ ]:

class_distribution = df_tweet['label'].value_counts()
plt.figure(figsize=(8, 6))
class_distribution.plot(kind='bar', color=['#4CAF50', '#F44336'], edgecolor='black')
plt.title('Class Distribution', fontsize=16)
plt.xlabel('Label', fontsize=14)
plt.ylabel('No of Tweets', fontsize=14)
plt.xticks(rotation=0, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add grid lines to y-axis with dashed style
plt.tight_layout()
plt.show()

In [ ]:
#============================ Tokenization ============================#

# Split into training, validation and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(clean_tweets, labels, test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

# Loading pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
#============================ Defining a PyTorch Dataset ============================#

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [ ]:
#============================ Loading the pre-trained Model ============================#

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Defining optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Defining data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Training loop
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

In [ ]:
from tqdm.auto import tqdm

for epoch in range(3):
    model.train()
    # Wrap the train_loader with tqdm for a progress bar
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    val_accuracy = 0
    # Wrap the val_loader with tqdm for a progress bar
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Validating Epoch {epoch + 1}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=1)
            val_accuracy += torch.sum(predictions == labels).item() / labels.size(0)

    avg_val_accuracy = val_accuracy / len(val_loader)
    print(f'Epoch {epoch + 1}, Validation Accuracy: {avg_val_accuracy}')

In [ ]:
#============================ Evaluating Model Performance ============================#

true_labels = []
predicted_labels = []

# Ensure model is on the correct device before evaluation
model.to(device)

# Evaluation loop
model.eval()
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predictions.cpu().numpy())

In [ ]:
# Confusion Matrix
cm = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, annot_kws={"size": 14})
plt.title('Confusion Matrix - BERT', fontsize = 16)
plt.xlabel('Predicted', fontsize = 14)
plt.ylabel('Actual', fontsize = 14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()


# Classification report
target_names = ['Appropriate', 'Inappropriate']
print('\nClassification Report:\n\n', classification_report(true_labels, predicted_labels, target_names=target_names))

# Evaluation Metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)
print("\nEvaluation Metrics:\n")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
#============================ Save the  model ============================#

# model.save_pretrained('/content/drive/MyDrive/text/bert_model/')
# model.save_pretrained('/bert_model/')

In [ ]:
#============================ Prediction ============================#

import torch
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification

#Text Preprocessing

stop_words = set(stopwords.words('english'))
stop_words.add("rt") # adding rt to remove retweet in dataset

# Removing Emojis
def remove_entity(raw_text):
    entity_regex = r"&[^\s;]+;"
    text = re.sub(entity_regex, "", raw_text)
    return text

# Replacing user tags
def change_user(raw_text):
    regex = r"@([^ ]+)"
    text = re.sub(regex, "", raw_text)
    return text

# Removing URLs
def remove_url(raw_text):
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text = re.sub(url_regex, '', raw_text)
    return text

# Removing Unnecessary Symbols
def remove_noise_symbols(raw_text):
    text = raw_text.replace('"', '')
    text = text.replace("'", '')
    text = text.replace("!", '')
    text = text.replace("`", '')
    text = text.replace("..", '')
    text = text.replace(".", '')
    text = text.replace(",", '')
    text = text.replace("#", '')
    text = text.replace(":", '')
    text = text.replace("?", '')
    return text

# Stemming
def stemming(raw_text):
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in raw_text.split()]
    return ' '.join(words)

# Removing stopwords
def remove_stopwords(raw_text):
    tokenize = word_tokenize(raw_text)
    text = [word for word in tokenize if not word.lower() in stop_words]
    text = ' '.join(text)
    return text

def preprocess(data):
    clean = []
    clean = [text.lower() for text in data]
    clean = [change_user(text) for text in clean]
    clean = [remove_entity(text) for text in clean]
    clean = [remove_url(text) for text in clean]
    clean = [remove_noise_symbols(text) for text in clean]
    clean = [stemming(text) for text in clean]
    clean = [remove_stopwords(text) for text in clean]

    return clean

In [ ]:
#============================ Load the saved model ============================#

from transformers import BertForSequenceClassification

loaded_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/text/bert_model', local_files_only=True)

# You can now use loaded_model for predictions or further evaluation
print("Model loaded successfully!")

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import matplotlib.pyplot as plt

# Load model and tokenizer
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/text/bert_model')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Your input
# custom_input_text = " now you're ready to vote for donald trump "

custom_input_text = "why the f**k"

# Optional: dummy preprocess function (replace if you have a custom one)
def preprocess(text_list):
    return text_list

custom_text_list = [custom_input_text]
preprocessed_custom_text = preprocess(custom_text_list)[0]

# Tokenization
tokenized_input = tokenizer(preprocessed_custom_text, return_tensors='pt')

# Prediction
model.eval()
with torch.no_grad():
    output = model(**tokenized_input)
    logits = output.logits
    probabilities = torch.softmax(logits, dim=1)

predicted_label = torch.argmax(probabilities, dim=1).item()
probability_class_0 = probabilities[0][0].item()
probability_class_1 = probabilities[0][1].item()
threshold = 0.4

if probability_class_1 >= threshold:
    print("⚠️ Prediction: Inappropriate (Offensive)")
else:
    print("✅ Prediction: Appropriate (Safe)")


# Print results
print("Predicted Label:", predicted_label)
print("Probability for Class 0 (Appropriate):", probability_class_0)
print("Probability for Class 1 (Inappropriate):", probability_class_1)

# ========================= 📊 Show Probability Bar Chart ========================= #
classes = ['Appropriate (Safe)', 'Inappropriate (Offensive)']
probs = [probability_class_0, probability_class_1]

plt.figure(figsize=(8, 2))
bars = plt.barh(classes, probs, color=['green', 'red'])
plt.xlim(0, 1)
plt.xlabel("Confidence")
plt.title(f"Prediction: {'Safe' if predicted_label == 0 else 'Offensive'}")

# Decide final prediction label based on threshold
threshold = 0.4
final_prediction = "Offensive" if probability_class_1 >= threshold else "Safe"

plt.title(f"Prediction: {final_prediction}")


# Add text on bars
for bar in bars:
    plt.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2,
             f"{bar.get_width()*100:.2f}%", va='center')

plt.tight_layout()
plt.show()
